# Пространственные преобразования изображения
## Радиальная дисторсия


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Зададим параметры калибровки для дальнейшего изучения

In [ ]:
K_akn = np.array(
    [[1.1801986200788842e+003, 0., 9.5996370019675282e+002],
     [0.,1.1801986200788842e+003, 5.7360826853365427e+002],
     [0., 0., 1.]],
    dtype=np.float32)
D_akn = np.array([-3.4670863170449812e-001, 1.9396306358433246e-001, 0., 0., -6.6846308891043940e-002],
            dtype=np.float32)
image_size_akn = (1920, 1080)

K_ksm = np.array(
      [[1.7074220094338809e+03, 0., 1.9301498561097922e+03],
       [0., 1.7074220094338809e+03, 1.0652992024104087e+03],
       [0., 0., 1.]],
      dtype=np.float32)
D_ksm = np.array([-2.6013055601960494e-01, 8.6016994641318525e-02, 0., 0.,-1.4475453463522846e-02], 
            dtype=np.float32)
image_size_ksm = (3840, 2160)

K = K_akn
D = D_akn
image_size = image_size_akn

print('K = ', K)
print('D = ', D)
print('image_size = ', image_size)

cap = cv2.VideoCapture('/testdata/akn/inp/akn.049/akn.049.001.left.avi')
# cap = cv2.VideoCapture('/testdata/ksm/inp/ksm.001/ksm.001.001.left.avi')

Будем изучать поведение преобразования на 

* сетке с константным шагом
* произвольно выбранных точках

Наша цель - изучить как и куда переходят точки при прямом и обратном преобразовании

In [ ]:
def draw_grid(img, step, color):
    w = img.shape[1]
    h = img.shape[0]
    start = int(step / 2)
    for i in range(start, w, step):
        cv2.line(img, (i, 0), (i, h - 1), color, 3, cv2.LINE_AA)
    for i in range(start, h, step):
        cv2.line(img, (0, i), (w - 1, i), color, 3, cv2.LINE_AA)
    
    return img

def generate_circles(image_size, n):
    cmap = plt.cm.get_cmap("tab20", n)
    xs = np.random.randint(0, image_size[0] - 1, n)
    ys = np.random.randint(0, image_size[1] - 1, n)
    cmis = np.random.permutation(n)
    colors = [cmap(cmi) for cmi in cmis]
    colors = [(255 * c[0], 255 * c[1], 255 * c[2]) for c in colors]
    return [{'x': x, 'y': y, 'color': color} for x,y,color in zip(xs, ys, colors)]

def draw_circles(image, circles):
    for c in circles:
        cv2.circle(image, (c['x'], c['y']), 20, c['color'], -1, cv2.LINE_AA)
    return image
        
def undistort_circles(circles, K, D):
    src = [(c['x'], c['y']) for c in circles]
    src = np.array(src, dtype=np.float32)
    src = src.reshape(-1,1,2)
    dst = cv2.undistortPoints(src, K, D, P=K)
    # print(src, dst)
    res = []
    for i,c in enumerate(circles):
        res.append({'x': dst[i,0,0], 'y': dst[i,0,1], 'color': c['color']})
    return res;
    
def prepare_distortion_grid_images(image_size, K, D):
    #img_src = np.zeros(image_size[-1::-1], dtype=np.uint8)
    img_src = 255 * np.ones((image_size[1], image_size[0], 3), dtype=np.uint8)
    img_src = draw_grid(img_src, step=100, color=(220, 100, 100))
    return img_src

In [ ]:
img_src = prepare_distortion_grid_images(image_size, K, D)
img_und = cv2.undistort(img_src, K, D)


circles_src = generate_circles(image_size, 20)
img_src_p = draw_circles(img_src.copy(), circles_src)
img_und_p = cv2.undistort(img_src_p, K, D)
circles_und = undistort_circles(circles_src, K, D)
img_und_p_2 = draw_circles(img_und.copy(), circles_und)

plt.figure(figsize=(20,13))
plt.subplot(221)
plt.imshow(img_src_p)
plt.title('src')
plt.subplot(222)
plt.imshow(img_und_p)
plt.title('undistort with src pts')
plt.subplot(223)
plt.imshow(img_src_p)
plt.title('src')
plt.subplot(224)
plt.imshow(img_und_p_2)
plt.title('undistort projected pts')
plt.tight_layout()
plt.show()

Не стоит путать картинки: слева (src) изображение с дисторсией. 
Справа - исправленная дисторсия. 

Искажение сетки показывает, куда сдвигаются пиксели исходного изображения при преобразовании

In [ ]:
_, src = cap.read()
und = cv2.undistort(src, K, D)
src_mask = img_src_p != 255
und_mask = img_und_p_2 != 255

und[und_mask] = img_und_p_2[und_mask]
src[src_mask] = img_src_p[src_mask]



In [ ]:
plt.figure(figsize=(20,13))
plt.subplot(211)
plt.imshow(src)
plt.title('src')
plt.subplot(212)
plt.imshow(und)
plt.title('undistorted')
plt.show()

Итак, мы продемонстрировали, как трансформируются точки из src в undistort. Но как узнать прообраз точки на und-изображении?

In [ ]:
def distortPoints(src, K, k1, k2, k3, P):
    n = len(src)
    homogenous_points = np.dstack([src, np.ones((n,1,1), dtype=np.float32)])
    h_p = homogenous_points.reshape(n,-1).T
    uniform_coords = np.linalg.inv(K) @ h_p
    xs = uniform_coords[0,:]
    ys = uniform_coords[1,:]
    
    r2 = xs ** 2 + ys ** 2
    r4 = r2 ** 2
    r6 = r4 * r2
    coef = (1 + k1 * r2 + k2 * r4 + k3 * r6)
    xs1 = xs * coef
    ys1 = ys * coef
    res = np.vstack([xs1, ys1, np.ones(n, dtype=np.float32)])
    res = P @ res
    res = res[:2].T.reshape(-1,1,2)
    return res

def distort_circles(circles, K, D):
    src = [(c['x'], c['y']) for c in circles]
    src = np.array(src, dtype=np.float32)
    src = src.reshape(-1,1,2)
    dst = distortPoints(src, K, D[0], D[1], D[4], P=K)
    res = []
    for i,c in enumerate(circles):
        res.append({'x': dst[i,0,0], 'y': dst[i,0,1], 'color': c['color']})
    return res;

In [ ]:
# circles = generate_circles(image_size, 20)
circles_distorted = distort_circles(circles_und, K, D)
img_src_p_2 = draw_circles(img_src.copy(), circles_distorted)
plt.figure(figsize=(20,13))
plt.subplot(121)
plt.imshow(img_src_p)
plt.title('initial circle position')
plt.subplot(122)
plt.imshow(img_src_p_2)
plt.title('distorted circles after undistortion')
plt.show()

p_src1 = np.array([[c['x'], c['y']] for c in circles_src], dtype=np.float32)
p_src2 = np.array([[c['x'], c['y']] for c in circles_distorted], dtype=np.float32)

print(p_src1 - p_src2)

### но как тогда работает функция cv2.undistortPoints?

необходимо обратить функцию `distortPoints`. 


$$
\left(
\begin{matrix}
x_{src} \\
y_{src}
\end{matrix}
\right)
=
\left(
\begin{matrix}
x_{dst} (1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\
y_{dst} (1 + k_1 r^2 + k_2 r^4 + k_3 r^6)
\end{matrix}
\right)
$$

где $r^2 = x_{dst}^2 + y_{dst}^2$


Воспользуемся пакетом символьных вычислений `sympy`

In [ ]:
class DistortFunc:
    def __init__(self, K, D):
        self.k1 = D[0]
        self.k2 = D[1]
        self.k3 = D[4]
        self.fx = K[0,0]
        self.fy = K[1,1]
        self.cx = K[0,2]
        self.cy = K[1,2]
        
        self.F_sym, self.J_sym = self.__symbols()
        self.FN_sym, self.JN_sym = self.__nsymbols()
    
    def F(self, xy):
        return np.array(self.F_sym.evalf(subs={self.x:xy[0], self.y:xy[1]}), dtype=np.float32)
    def J(self, xy):
        return np.array(self.J_sym.evalf(subs={self.x:xy[0], self.y:xy[1]}), dtype=np.float32)
    
    def Fn(self, xy):
        return np.array(self.FN_sym.evalf(subs={self.x:xy[0], self.y:xy[1]}), dtype=np.float32)
    def Jn(self, xy):
        return np.array(self.JN_sym.evalf(subs={self.x:xy[0], self.y:xy[1]}), dtype=np.float32)
    
    def distort_pix(self, x, y):
        x1 = (x - self.cx) / self.fx
        y1 = (y - self.cy) / self.fy
        
        xy2 = self.F([x1,y1])
        x3 = self.fx * xy2[0][0] + self.cx
        y3 = self.fy * xy2[1][0] + self.cy
        return (x3, y3)
    
    def normalize_x(self, x):
        return (x - self.cx) / self.fx
    def normalize_y(self, y):
        return (y - self.cy) / self.fy   
    def unnormalize_x(self, x):
        return self.fx * x + self.cx
    def unnormalize_y(self, y):
        return self.fy * y + self.cy
    
    
    def __symbols(self):
        import sympy as sym

        self.x, self.y = sym.symbols('x, y')
        
        def r2(x, y):
            return x * x + y * y

        def d(x, y):
            r2_ = r2(x, y)
            r4_ = r2_ * r2_
            r6_ = r4_ * r2_
            return 1 + self.k1 * r2_ + self.k2 * r4_ + self.k3 * r6_

        def fx(x, y):
            return x * d(x, y)

        def fy(x, y):
            return y * d(x, y)

        F = sym.Matrix([fx(self.x, self.y), fy(self.x, self.y)])
        J = F.jacobian((self.x, self.y))
        return F,J
    
    def __nsymbols(self):
        import sympy as sym

        self.x, self.y = sym.symbols('x, y')
        
        def r2(x, y):
            return x * x + y * y

        def d(x, y):
            r2_ = r2(x, y)
            r4_ = r2_ * r2_
            r6_ = r4_ * r2_
            return 1 + self.k1 * r2_ + self.k2 * r4_ + self.k3 * r6_

        def fx(x, y):
            return x * d(x, y)

        def fy(x, y):
            return y * d(x, y)
        
        nx = self.normalize_x(self.x)
        ny = self.normalize_y(self.y)
        
        xx = fx(nx, ny)
        yy = fy(nx, ny)
        xxx = self.unnormalize_x(xx)
        yyy = self.unnormalize_y(yy)
        
        F = sym.Matrix([xxx, yyy])
        J = F.jacobian((self.x, self.y))
        return F,J        
        

In [ ]:
circles_src[1:5], circles_und[1:5]

In [ ]:
df = DistortFunc(K, D)
print(df.distort_pix(1819.3844, 494.98703))
print(df.distort_pix(2081.6938, 872.96783))
print(df.distort_pix(-178.85303, 307.56543))


import scipy.optimize

def undistortPoint(xy_, cb = None, method="lm"):
    def func(xy):
        return df.Fn(xy).ravel() - xy_
    def jac(xy):
        return df.Jn(xy)
    root = scipy.optimize.root(func, xy_, jac=jac, callback=cb, method=method)
    #print(root)
    return root.x

print(undistortPoint([1699, 506]))
print(undistortPoint([1842, 809]))
print(undistortPoint([67, 365]))

###  пронаблюдаем как происходит сходимость алгоиртма

In [ ]:
class OptStats:
    def __init__(self, circles):
        self.cirlcles = circles
        self.N = len(circles)
        self.pthist={}
    def get_opt_cb(self, icircle):
        self.pthist[icircle] = []
        def cb(x,f):
            self.pthist[icircle].append(x)
        return cb

stats = OptStats(circles_src)
und_pts = []
for i,c in enumerate(circles_src):
    und_pt = undistortPoint([c['x'], c['y']], cb=stats.get_opt_cb(i), method='anderson')
    und_pts.append(und_pt)

In [ ]:

img_und_p_3 = draw_circles(img_und.copy(), circles_und)
img_ind_p_3 = draw_circles(img_und_p_3, circles_src)
for i, c in enumerate(circles_src):
    pp = tuple(np.rint(und_pts[i]).astype(np.int32))
    h = stats.pthist[i]
        
    cv2.circle(img_und_p_3, pp, 10, (20,220,20), -1, cv2.LINE_AA)
    
    for p in h[-2::-1]:
        p = tuple(np.rint(p).astype(np.int32))
        cv2.line(img_und_p_3, pp, p, c['color'], 2, cv2.LINE_AA)
        cv2.circle(img_und_p_3, p, 6, (90, 250, 90), -1, cv2.LINE_AA)
        pp = p
    cv2.circle(img_und_p_3, p, 10, (20,20,220), -1, cv2.LINE_AA)


plt.figure(figsize=(15,20))
plt.subplot(211)
plt.imshow(img_src_p)
plt.subplot(212)
plt.imshow(img_und_p_3)
plt.show()